# Web scraping the top 3 advanced statistical metrics in basketball.

*The last code review date: March 12, 2024.*

## Introduction

Traditional statistical metrics in basketball often fail to provide a comprehensive reflection of reality on the court. While a player may exhibit a stellar offensive performance, sinking shots with high accuracy, defensive lapses could result in the team conceding more points than the player's offensive contributions. To address this gap, advanced statistical metrics have emerged, aiming to offer a nuanced understanding of player impact on team performance. In this analysis, we delve into three cutting-edge metrics highly regarded in the basketball community: LEBRON, EPM, and DARKO.

### LEBRON
- **Full Title:** Luck-adjusted Player Estimate using a Box prior Regularized ON-off
- **Concept:** This metric leverages a player's influence on team Plus-minus, employing weighted boxPIPM coefficients and RAPM methodology to provide a holistic assessment of their impact on the game, with adjustments made for luck factors.

### EPM
- **Full Title:** Estimated Plus-Minus
- **Concept:** EPM evaluates a player's impact on the team's performance when they're on and off the court, incorporating Bayesian prior values to quantify their contribution.

### DARKO
- **Full Title:** Daily Plus-Minus
- **Concept:** DARKO introduces a machine learning-based scoring prediction system for basketball players, integrating time series data and sample size considerations. Its Bayesian model forecasts all aspects of box score estimation, showcasing a player's projected skill development over time. Notably, DARKO surpasses other publicly available indicators in predictive accuracy, as measured by root mean square error (RMSE).

These metrics offer distinct methodologies for assessing a player's influence on team performance. The ensuing table presents player ratings across each metric, culminating in an average score. By examining the player's ranking in each metric—LEBRON, EPM, and DARKO—and calculating the average, we minimize potential evaluation discrepancies inherent in single-model approaches. This consolidated ranking, represented in the RANK_TOP3_AVG column, serves as a reliable benchmark for gauging player performance based on advanced metrics.

This metric not only caters to basketball enthusiasts seeking a deeper understanding of player dynamics but also provides valuable insights for industry experts navigating the intricacies of on-court performance analysis and prediction.

In [1]:
import math  # Importing math for mathematical functions
import datetime  # Importing datetime for working with dates and times
import warnings  # Importing warnings for handling warning messages
import time  # Importing time for working with time-related functions

import requests  # Importing the requests module for sending HTTP requests
from bs4 import BeautifulSoup  # Importing BeautifulSoup from bs4 for parsing HTML and XML documents
import pandas as pd  # Importing pandas for data manipulation and analysis
from functools import reduce  # Importing the 'reduce' function from the 'functools' module.

import selenium.webdriver  # Importing WebDriver module from Selenium for web automation
from selenium import webdriver # Importing the webdriver module from Selenium
from selenium.webdriver.common.by import By  # Importing the By class from Selenium for locating elements by various strategies
from selenium.webdriver.common.keys import Keys  # Importing the Keys class from Selenium for keyboard actions
from selenium.webdriver.support.ui import WebDriverWait  # Importing WebDriverWait from Selenium for waiting for certain conditions
from selenium.webdriver.support import expected_conditions as EC  # Importing expected_conditions from Selenium for defining expected conditions

from io import StringIO  # Importing StringIO for reading and writing strings as files
from lxml import etree  # Importing etree from lxml for processing XML and HTML

In [2]:
# Suppressing SettingWithCopyWarning warnings
pd.options.mode.chained_assignment = None  # default='warn'

# Set the warning level to "ignore"
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## DARKO (Daily Plus-Minus)

In [4]:
def scrape_data_DARKO(number):
    """
    Scrapes data from a webpage using Selenium WebDriver.

    Args:
    number (int): Number of times to click the "Next" button to scrape data from multiple pages.

    Returns:
    pandas.DataFrame: Scraped data.

    """
    # Set up Chrome options
    options = webdriver.ChromeOptions()
    options.add_argument('headless')  # Run Chrome in the Chrome Headless Mode

    # Use a context manager for the Chrome WebDriver
    with webdriver.Chrome(options=options) as driver:
        wait = WebDriverWait(driver, 12)  # Set up WebDriverWait

        url_DARKO = 'https://apanalytics.shinyapps.io/DARKO/'
        driver.get(url_DARKO)  # Open the specified URL

        # Click on the "Current Player Skill Projections" tab
        current_player_skill_projections_tab = wait.until(EC.presence_of_element_located((By.LINK_TEXT, "Current Player Skill Projections")))
        current_player_skill_projections_tab.click()

        start_time = time.time()  # Record the start time
        while time.time() - start_time < 5:
            pass  # Wait for 5 seconds for the page to load

        # Get the HTML source after clicking the tab
        html_on_Current_Player_Skill_Projections = driver.page_source
        soup_after_click = BeautifulSoup(html_on_Current_Player_Skill_Projections, "html.parser")

        # Find the table with id "DataTables_Table_0"
        table = soup_after_click.find('table', {'id': 'DataTables_Table_0'})

        # Read the table into a DataFrame
        df = pd.read_html(str(table))[0]
        
        # Loop to click on the "Next" button and scrape data from multiple pages
        for _ in range(number):
            next_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "paginate_button.next")))
            next_button.click()

            start_time = time.time()
            while time.time() - start_time < 3:
                pass  # Wait for 3 seconds

            # Get the HTML source after clicking the "Next" button
            html_after_next_click = driver.page_source
            soup_after_next_click = BeautifulSoup(html_after_next_click, "html.parser")

            # Find the table after clicking the "Next" button and convert it to a DataFrame
            table_after_next_click = soup_after_next_click.find('table', {'id': 'DataTables_Table_0'})
            df_next = pd.read_html(str(table_after_next_click))[0]
            df = pd.concat([df, df_next], ignore_index=True)

    return df

# Call the scrape_data function with a specified number of iterations
df_DARKO = scrape_data_DARKO(55)

# Drop rows with all NaN values
df_DARKO = df_DARKO.dropna(how='all')

# Drop duplicate rows
df_DARKO = df_DARKO.drop_duplicates()

# Convert the 'DPM' column to float
df_DARKO['DPM'] = df_DARKO['DPM'].astype(float)

# Add a 'DPM_RANK' column with the rank of values in the 'DPM' column
df_DARKO['DPM_RANK'] = df_DARKO['DPM'].rank(ascending=False)

# Convert the 'DPM_RANK' column to integer
df_DARKO['DPM_RANK'] = df_DARKO['DPM_RANK'].astype(int)

## EPM (Estimated Plus-Minus)

In [5]:
def scrape_epm_data_epm():
    """
    Scrape data from the EPM website.

    Returns:
        pandas.DataFrame: Scraped data.
    """
    # Browser options (for running the browser with the Chrome Headless Mode)
    options = webdriver.ChromeOptions()
    options.add_argument('headless')

    # Initializing the browser driver
    driver = webdriver.Chrome(options=options)

    # Website URL
    url_epm = 'https://dunksandthrees.com/epm'

    # Opening the website
    driver.get(url_epm)

    # Waiting for the table to load
    table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.table-wrap table.main')))

    # Extracting data from the first 10 rows
    rows_data_epm = []
    for row in table.find_elements(By.CSS_SELECTOR, 'tr.svelte-blwps1')[:10]:
        cells = row.find_elements(By.CLASS_NAME, 'svelte-blwps1')
        row_data_epm = [cell.text for cell in cells]
        rows_data_epm.append(row_data_epm)

    # Scrolling the table and extracting data after each scroll
    scroll_pixels = 360
    row_height = 36
    for scroll_count in range(1, 48):  # Scroll 48 times
        # Execute JavaScript to scroll the table
        driver.execute_script(f"document.querySelector('div.table-wrap').scrollTop += {scroll_pixels}")
        # Wait for new data to load
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.loading')))

        # Extract data from table cells
        for row in table.find_elements(By.CSS_SELECTOR, 'tr.svelte-blwps1')[scroll_count*10:scroll_count*10+10]:
            cells = row.find_elements(By.CLASS_NAME, 'svelte-blwps1')
            row_data_epm = [cell.text for cell in cells]
            rows_data_epm.append(row_data_epm)

    # Close the browser
    driver.quit()

    # Remove the first two nested lists
    rows_data_epm = rows_data_epm[2:]

    # Convert to a DataFrame without column names
    df_epm = pd.DataFrame(rows_data_epm)

    return df_epm

# Scraping data using a function named scrape_epm_data_epm() and assigning the result to the DataFrame df_epm
df_epm = scrape_epm_data_epm()

In [6]:
def shift_rows(df, condition, num_columns_to_shift):
    """
    Shifts rows in a DataFrame based on the given condition and number of columns to shift.
    
    Parameters:
        df (DataFrame): The DataFrame to operate on.
        condition (Series): The condition to filter rows.
        num_columns_to_shift (int): The number of columns to shift.
    
    Returns:
        DataFrame: The DataFrame with shifted rows.
    """
    # Filtering rows based on condition to get subset of rows
    filtered_rows_1 = df_epm[df_epm[31].isnull()]
    filtered_rows_2 = filtered_rows_1[condition]

    # Checking if both original and filtered subsets are not empty
    if not filtered_rows_1.empty and not filtered_rows_2.empty:
        # Iterating through rows of filtered subset and shifting values to the right
        for index, row in filtered_rows_2.iterrows():
            for i in range(num_columns_to_shift, len(row) - 2):
                df_epm.loc[index, i + 2] = row[i]
    
    return df_epm

# Apply the function to shift rows for players without 3p% based on specified conditions
df_epm = shift_rows(df_epm, (df_epm[31].isnull()) & 
                            (df_epm[13] != '') & 
                            (df_epm[16] != '') & 
                            (df_epm[19] == ''), 20)

# Apply function with condition 2 for players without FT% based on specified conditions
df_epm = shift_rows(df_epm, (df_epm[31].isnull()) & 
                            (df_epm[13] != '') & 
                            (df_epm[19] != '') & 
                            (df_epm[22] == ''), 23)
# Apply the function to shift rows for players without mid% based on specified conditions
df_epm = shift_rows(df_epm, (df_epm[31].isnull()) & 
                            (df_epm[13] != '') & 
                            (df_epm[17] != '') & 
                            (df_epm[16] == ''), 18)
# Apply the function to shift rows for players without mid% and 3p% based on specified conditions
df_epm = shift_rows(df_epm, (df_epm[31].isnull()) & 
                            (df_epm[13] != '') & 
                            (df_epm[16] == '') & 
                            (df_epm[17] == '') & 
                            (df_epm[19] == ''), 20)
# Apply the function to shift rows for players without mid%, 3p% and FT% based on specified conditions
df_epm = shift_rows(df_epm, (df_epm[31].isnull()) & 
                            (df_epm[13] != '') & 
                            (df_epm[16] == '') & 
                            (df_epm[17] == '') & 
                            (df_epm[19] == '') & 
                            (df_epm[20] == ''), 20)
# Apply the function to shift rows for players without rimn %, mid% and FT% based on specified conditions
df_epm = shift_rows(df_epm, (df_epm[31].isnull()) & 
                            (df_epm[13] == '') & 
                            (df_epm[16] == '') & 
                            (df_epm[17] == '') & 
                            (df_epm[19] != '') & 
                            (df_epm[23] != ''), 23)

/var/folders/t2/jpft56jd66n3rmt8dqdn1dlm0000gn/T/ipykernel_5224/11592769.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows_2 = filtered_rows_1[condition]
/var/folders/t2/jpft56jd66n3rmt8dqdn1dlm0000gn/T/ipykernel_5224/11592769.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows_2 = filtered_rows_1[condition]
/var/folders/t2/jpft56jd66n3rmt8dqdn1dlm0000gn/T/ipykernel_5224/11592769.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows_2 = filtered_rows_1[condition]
/var/folders/t2/jpft56jd66n3rmt8dqdn1dlm0000gn/T/ipykernel_5224/11592769.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows_2 = filtered_rows_1[condition]
/var/folders/t2/jpft56jd66n3rmt8dqdn1dlm0000gn/T/ipykernel_5224/11592769.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_rows_2 = filtered_row

In [7]:
# Specify the column numbers to be dropped
columns_epm_to_drop = [11, 12, 14, 15, 17, 18, 20, 21, 23, 24]

# Drop columns by the specified indexes
df_epm.drop(columns=columns_epm_to_drop, inplace=True)

# Specify the column names
column_names_epm = ['Player_all_info', 'Player', 'GP', 'MPG', 'USG', '+/- OFF_EPM', '+/- DEF_EPM', 
                'EPM', 'Estimated_wins', 'TS%', 'eFG%', 'FG at te rim', 'FG% fron midrannge', 
                '3PT%', 'FT%', 'OR%', 'DR%', 'AST%', 'TO%', 'ST%', 'BL%', 'Rank_EPM']

# Set the column names
df_epm.columns = column_names_epm

# Split the "Player_all_info" column into multiple columns
df_epm[['Player', 'Team', 'Position', 'Age']] = df_epm['Player_all_info'].str.split('\n|·', expand=True)[[0, 1, 2, 3]]

# List of columns to process
columns_to_process = ['GP', 'MPG', 'USG', '+/- OFF_EPM', '+/- DEF_EPM', 
                      'EPM', 'Estimated_wins', 'TS%', 'eFG%', 'FG at te rim', 
                      'FG% fron midrannge', '3PT%', 'FT%', 'OR%', 'DR%', 
                      'AST%', 'TO%', 'ST%', 'BL%']

# Apply the operation to split and select the first part for each value in the specified columns
df_epm[columns_to_process] = df_epm[columns_to_process].apply(lambda x: x.str.split('\n').str[0])

# New order of columns
new_column_order = ['Player', 'Team', 'Position', 'Age', 'GP', 'MPG', 'USG', '+/- OFF_EPM', '+/- DEF_EPM', 
                    'EPM', 'Estimated_wins', 'TS%', 'eFG%', 'FG at te rim', 'FG% fron midrannge', 
                    '3PT%', 'FT%', 'OR%', 'DR%', 'AST%', 'TO%', 'ST%', 'BL%', 'Rank_EPM']

# Select and reorder columns
df_epm = df_epm[new_column_order]

# Convert empty strings to NaN and convert values to numeric format
columns_to_convert_float = ['MPG', 'USG', 'TS%', 'eFG%', 'FG at te rim', 'FG% fron midrannge', 
                            '3PT%', 'FT%', 'OR%', 'DR%', 'AST%', 'TO%', 'ST%', 'BL%']
df_epm[columns_to_convert_float] = df_epm[columns_to_convert_float].apply(pd.to_numeric, errors='coerce')

# Reset index and adjust index values
df_epm.reset_index(drop=True, inplace=True)
df_epm.index += 1

# Convert 'Rank_Lebron' column to integer data type
df_epm['Rank_EPM'] = df_epm['Rank_EPM'].astype(int)

## LEBRON (Luck-adjusted player Estimate using a Box prior Regularized ON-off)

In [ ]:
# URL of the page to be parsed
url_lebron = 'https://www.bball-index.com/lebron-database/'

# Send a GET request to the specified URL and save the page content
response_lebron = requests.get(url_lebron)

soup_lebron = BeautifulSoup(response_lebron.content, 'html.parser')
table_lebron = soup_lebron.find('table', {'id': 'table_1'})

data_lebron = []
for row in table_lebron.find_all('tr'):
   cols = row.find_all(['th', 'td'])
   cols = [ele.text.strip() for ele in cols]
   data_lebron.append([ele for ele in cols if ele])

# Creating DataFrame using Pandas
df_lebron = pd.DataFrame(data_lebron[1:], columns=data_lebron[0])

In [ ]:
# Drop unnecessary columns and filter out rows with specific values
df_lebron = df_lebron.drop(['bref_id', 'NBA ID'], axis=1).query('Minutes != "Pos" and Age != "-0.30"')

# Convert selected columns to numeric data type
columns_to_convert_float_lebron = ['LEBRON', 'O-LEBRON', 'D-LEBRON', 'boxLEBRON', 'boxOLEBRON', 'boxDLEBRON']
df_lebron[columns_to_convert_float_lebron] = df_lebron[columns_to_convert_float_lebron].apply(pd.to_numeric, errors='coerce')

# Convert a column to float after removing commas
df_lebron["LEBRON Contract Value"] = df_lebron["LEBRON Contract Value"].str.replace(',', '').astype(float)

# Filter rows for the 2023-24 season
df_lebron_2023_2024 = df_lebron[df_lebron['Season'] == '2023-24']

# Create a dictionary to store last year's LEBRON values for each player
lebron_last_year_dict_LEBRON = {}
for player in df_lebron_2023_2024['Player'].unique():
    player_row = df_lebron[(df_lebron['Player'] == player) & (df_lebron['Season'] == '2022-23')]
    if not player_row.empty:
        lebron_last_year_dict_LEBRON[player] = player_row.iloc[0]['LEBRON']
    else:
        lebron_last_year_dict_LEBRON[player] = float('nan')

# Map last year's LEBRON values to the DataFrame
df_lebron_2023_2024.loc[:, 'LEBRON_last_year'] = df_lebron_2023_2024['Player'].map(lebron_last_year_dict_LEBRON)

# Create dictionaries to store last year's O-LEBRON and D-LEBRON values for each player
lebron_last_year_dicts = {'O-LEBRON': {}, 'D-LEBRON': {}}
for player in df_lebron_2023_2024['Player'].unique():
    for lebron_type in ['O-LEBRON', 'D-LEBRON']:
        player_row = df_lebron[(df_lebron['Player'] == player) & (df_lebron['Season'] == '2022-23')]
        if not player_row.empty:
            lebron_last_year_dicts[lebron_type][player] = player_row.iloc[0][lebron_type]
        else:
            lebron_last_year_dicts[lebron_type][player] = float('nan')

# Map last year's O-LEBRON and D-LEBRON values to the DataFrame
for lebron_type in ['O-LEBRON', 'D-LEBRON']:
    df_lebron_2023_2024.loc[:, f'{lebron_type}_last_year'] = df_lebron_2023_2024['Player'].map(lebron_last_year_dicts[lebron_type])

# Calculate rank based on LEBRON values
df_lebron_2023_2024['Rank_Lebron'] = df_lebron_2023_2024['LEBRON'].rank(ascending=False, method='min')

# Reorder columns in the DataFrame
new_order_2023_2024 = ['Player', 'Season', 'Team', 'Rank_Lebron', 'Offensive Archetype', 'Minutes', 'Games', 'Pos', 'Age', 'LEBRON', 'LEBRON_last_year', 'O-LEBRON',
                       'O-LEBRON_last_year', 'D-LEBRON', 'D-LEBRON_last_year', 'WAR', 'LEBRON Contract Value', 'boxLEBRON', 'boxOLEBRON', 'boxDLEBRON']
df_lebron_2023_2024 = df_lebron_2023_2024.reindex(columns=new_order_2023_2024)

# Sort DataFrame by LEBRON values
df_lebron_2023_2024 = df_lebron_2023_2024.sort_values(by='LEBRON', ascending=False)

# Reset index of the DataFrame
df_lebron_2023_2024.reset_index(drop=True, inplace=True)
df_lebron_2023_2024.index += 1

# Convert 'Rank_Lebron' column to integer data type
df_lebron_2023_2024['Rank_Lebron'] = df_lebron_2023_2024['Rank_Lebron'].astype(int)


## Creating a unified table for three metrics

In [ ]:
# Merging all DataFrames at once
merged_df = df_epm.merge(df_DARKO, on='Player', how='left')
df_top_3_stats = merged_df.merge(df_lebron_2023_2024, on='Player', how='left')

# Convert 'Rank_Lebron' column to integer data type
df_epm['Rank_EPM'] = df_epm['Rank_EPM'].astype(int)

# Calculate the average of the ranks for 'Rank_EPM', 'Rank_Lebron', and 'DPM_RANK' columns.
# Add a new column 'RANK_TOP3_AVG' to store these average values.
df_top_3_stats['RANK_TOP3_AVG'] = df_top_3_stats[['Rank_EPM', 'Rank_Lebron', 'DPM_RANK']].mean(axis=1)

# Sort the DataFrame 'df_top_3_stats' based on the values in the 'RANK_TOP3_AVG' column in ascending order.
df_top_3_stats = df_top_3_stats.sort_values(by='RANK_TOP3_AVG')

# Removing columns from the DataFrame
columns_to_drop = ['Team_x', 'Experience', 'Season', 'Team', 'Minutes', 'Games', 'Pos', 'Age_y', 'FG at te rim', 'FG% fron midrannge', '3PT%', 'FT%_x', 'OR%', 'DR%', 'AST%_x', 'TO%', 'ST%', 'BL%', 'USG%']
df_top_3_stats.drop(columns=columns_to_drop, inplace=True)

# Calculating the mean of three ranking columns and creating a new column for the average rank
df_top_3_stats['RANK_TOP3_AVG'] = df_top_3_stats[['Rank_EPM', 'Rank_Lebron', 'DPM_RANK']].mean(axis=1)
df_top_3_stats= df_top_3_stats.sort_values(by='RANK_TOP3_AVG')

# Calculating improvements for LeBron-related stats compared to last year
df_top_3_stats['LEBRON_Improvement'] = df_top_3_stats['LEBRON'] - df_top_3_stats['LEBRON_last_year']
df_top_3_stats['LEBRON_Improvement_O'] = df_top_3_stats['O-LEBRON'] - df_top_3_stats['O-LEBRON_last_year']
df_top_3_stats['LEBRON_Improvement_D'] = df_top_3_stats['D-LEBRON'] - df_top_3_stats['D-LEBRON_last_year']

desired_columns = [
    'Player', 'Position', 'Offensive Archetype', 'Team_y', 'Age_x', 'GP', 'MPG', 'USG', 'RANK_TOP3_AVG',
    'Rank_EPM', 'DPM_RANK', 'Rank_Lebron', 'LEBRON_Improvement', 'LEBRON_Improvement_O', 'LEBRON_Improvement_D', 'DPM Improvement',
    '+/- OFF_EPM', '+/- DEF_EPM', 'EPM', 'Estimated_wins', 'DPM', 'O-DPM', 'D-DPM', 'Box DPM', 'Box O-DPM', 'Box D-DPM',
    'LEBRON', 'LEBRON_last_year', 'O-LEBRON', 'O-LEBRON_last_year', 'D-LEBRON', 'D-LEBRON_last_year', 'WAR',
    'LEBRON Contract Value', 'boxLEBRON', 'boxOLEBRON', 'boxDLEBRON', 'TS%', 'eFG%', 'FGA/100', 'FG2%', 'FG3A/100',
    'FG3%', 'FG3ARate%', 'RimFGA/100', 'RimFG%', 'FTA/100', 'FT%_y', 'FTARate%', 'REB/100', 'AST/100', 'AST%_y',
    'BLK/100', 'BLK%', 'STL/100', 'STL%', 'TOV/100'
]

# Selecting desired columns from DataFrame df_top_3_stats
df_top_3_stats = df_top_3_stats[desired_columns]

# Renaming columns of DataFrame
df_top_3_stats = df_top_3_stats.rename(columns={'Team_y': 'Team', 'Age_x': 'Age', 'FT%_y': 'FT%', 'AST%_y': 'AST%'})

# Defining the second-level columns
columns = pd.MultiIndex.from_tuples([
    ('Player', 'Player'),
    ('Player_info', 'Position'),
    ('Player_info', 'Offensive Archetype'),
    ('Player_info', 'Team'),
    ('Player_info', 'Age'),
    ('Player_info', 'GP'),
    ('Player_info', 'MPG'),
    ('Player_info', 'USG'),
    ('Advanced_stats', 'RANK_TOP3_AVG'),
    ('Advanced_stats', 'Rank_EPM'),
    ('Advanced_stats', 'DPM_RANK'),
    ('Advanced_stats', 'Rank_Lebron'),
    ('Improve_last_year', 'LEBRON_Improvement'),
    ('Improve_last_year', 'LEBRON_Improvement_O'),
    ('Improve_last_year', 'LEBRON_Improvement_D'),
    ('Improve_last_year', 'DPM Improvement'),
    ('EPM', '+/- OFF_EPM'),
    ('EPM', '+/- DEF_EPM'),
    ('EPM', 'EPM'),
    ('EPM', 'Estimated_wins'),
    ('DARKO', 'DPM'),
    ('DARKO', 'O-DPM'),
    ('DARKO', 'D-DPM'),
    ('DARKO', 'Box DPM'),
    ('DARKO', 'Box O-DPM'),
    ('DARKO', 'Box D-DPM'),
    ('Lebron_stats', 'LEBRON'),
    ('Lebron_stats', 'LEBRON_last_year'),
    ('Lebron_stats', 'O-LEBRON'),
    ('Lebron_stats', 'O-LEBRON_last_year'),
    ('Lebron_stats', 'D-LEBRON'),
    ('Lebron_stats', 'D-LEBRON_last_year'),
    ('Lebron_stats', 'WAR'),
    ('Lebron_stats', 'LEBRON Contract Value'),
    ('Lebron_stats', 'boxLEBRON'),
    ('Lebron_stats', 'boxOLEBRON'),
    ('Lebron_stats', 'boxDLEBRON'),
    ('Basic_Stats', 'TS%'),
    ('Basic_Stats', 'eFG%'),
    ('Basic_Stats', 'FGA/100'),
    ('Basic_Stats', 'FG2%'),
    ('Basic_Stats', 'FG3A/100'),
    ('Basic_Stats', 'FG3%'),
    ('Basic_Stats', 'FG3ARate%'),
    ('Basic_Stats', 'RimFGA/100'),
    ('Basic_Stats', 'RimFG%'),
    ('Basic_Stats', 'FTA/100'),
    ('Basic_Stats', 'FT%'),
    ('Basic_Stats', 'FTARate%'),
    ('Basic_Stats', 'REB/100'),
    ('Basic_Stats', 'AST/100'),
    ('Basic_Stats', 'AST%'),
    ('Basic_Stats', 'BLK/100'),
    ('Basic_Stats', 'BLK%'),
    ('Basic_Stats', 'STL/100'),
    ('Basic_Stats', 'STL%'),
    ('Basic_Stats', 'TOV/100')
])

# Renaming DataFrame columns using multi-index
df_top_3_stats.columns = columns


In [ ]:
df_top_3_stats

## Saving the table (if needed)

In [ ]:
# Get the current date
current_date = datetime.datetime.now().strftime("%Y-%m-%d")

# Create a file name with the current date for Excel format
file_name_excel = f"top_3_basket_stats_{current_date}.xlsx"

# Create a file name with the current date for CSV format
file_name_csv = f"top_3_basket_stats_{current_date}.csv"

In [ ]:
# Save the DataFrame to a CSV file with the specified file name and path. If you are using the code, please specify the actual file path
# df_top_3_stats.to_csv(f"/Users/andrejviflyancev/Desktop/{file_name_csv}", index=False)

# Save the DataFrame to an Excel file with the specified file name and path. If you are using the code, please specify the actual file path
# df_top_3_stats.to_excel(f"/Users/andrejviflyancev/Desktop/{file_name_excel}", index=False)